In [ ]:
import os
project_name = "recobase"; branch = "US987772"; account = "recohut"
project_path = os.path.join('/content', branch)

if not os.path.exists(project_path):
    !pip install -U -q dvc dvc[gdrive]
    !cp -r /content/drive/MyDrive/git_credentials/. ~
    !mkdir "{project_path}"
    %cd "{project_path}"
    !git init
    !git remote add origin https://github.com/"{account}"/"{project_name}".git
    !git pull origin "{branch}"
    !git checkout -b "{branch}"
    %reload_ext autoreload
    %autoreload 2
else:
    %cd "{project_path}"

/content/US987772


In [ ]:
import glob

glob.glob('./data/silver/**/*', recursive=True)

['./data/silver/ml-1m',
 './data/silver/taobao',
 './data/silver/processed',
 './data/silver/raw',
 './data/silver/book-crossing',
 './data/silver/ml-1m/users.csv.dvc',
 './data/silver/ml-1m/split_data_backup',
 './data/silver/ml-1m/feature_dict.pkl',
 './data/silver/ml-1m/user_dict.pkl',
 './data/silver/ml-1m/item_dict.pkl',
 './data/silver/ml-1m/ratings.csv.dvc',
 './data/silver/ml-1m/implicit_ratings.csv',
 './data/silver/ml-1m/movies.csv.dvc',
 './data/silver/ml-1m/split_data_backup/train_data.csv',
 './data/silver/ml-1m/split_data_backup/test_data.csv',
 './data/silver/ml-1m/split_data_backup/valid_data.csv',
 './data/silver/taobao/feature_dict.pkl',
 './data/silver/taobao/user_dict.pkl',
 './data/silver/taobao/item_dict.pkl',
 './data/silver/taobao/implicit_ratings.csv',
 './data/silver/processed/ml-1m',
 './data/silver/processed/pre_filter.pt',
 './data/silver/processed/pre_transform.pt',
 './data/silver/processed/book-crossing',
 './data/silver/processed/ml-1m/ml-1m.dataset',
 

In [ ]:
!rm ./data/silver/processed/book-crossing/book-crossing.statinfo

In [ ]:
!git status -u

In [21]:
!git add .
!git commit -m 'commit'
!git push origin "{branch}"

[US987772 b2bc675] commit
 47 files changed, 776 insertions(+), 48 deletions(-)
 create mode 100644 __pycache__/config.cpython-37.pyc
 create mode 100644 config.py
 create mode 100644 data/silver/book-crossing/.gitignore
 create mode 100644 data/silver/book-crossing/feature_dict.pkl.dvc
 create mode 100644 data/silver/book-crossing/implicit_ratings.csv.dvc
 create mode 100644 data/silver/book-crossing/item_dict.pkl.dvc
 create mode 100644 data/silver/book-crossing/user_dict.pkl.dvc
 create mode 100644 data/silver/ml-1m/feature_dict.pkl.dvc
 create mode 100644 data/silver/ml-1m/implicit_ratings.csv.dvc
 create mode 100644 data/silver/ml-1m/item_dict.pkl.dvc
 create mode 100644 data/silver/ml-1m/user_dict.pkl.dvc
 create mode 100644 data/silver/taobao/.gitignore
 create mode 100644 data/silver/taobao/feature_dict.pkl.dvc
 create mode 100644 data/silver/taobao/implicit_ratings.csv.dvc
 create mode 100644 data/silver/taobao/item_dict.pkl.dvc
 create mode 100644 data/silver/taobao/user_dict

In [ ]:
%%writefile requirements.txt
icecream
torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
torch-geometric
wget
libarchive

Overwriting requirements.txt


In [ ]:
!pip install -r requirements.txt

## Data

In [ ]:
import torch
import random
import numpy as np
import time
import argparse
from sklearn.model_selection import train_test_split
from torch_geometric.data import DataLoader

from src.datasets import GMCFDataset
from src.trainers import GMCFTrainer

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='ml-1m', help='which dataset to use')
parser.add_argument('--rating_file', type=str, default='implicit_ratings.csv', help='which dataset to use')
parser.add_argument('--dim', type=int, default=64, help='dimension of entity and relation embeddings')
parser.add_argument('--l2_weight', type=float, default=1e-5, help='weight of the l2 regularization term')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--batch_size', type=int, default=1024, help='batch size')
parser.add_argument('--n_epoch', type=int, default=50, help='the number of epochs')
parser.add_argument('--hidden_layer', type=int, default=256, help='neural hidden layer')
parser.add_argument('--num_user_features', type=int, default=3, help='the number of user attributes')
parser.add_argument('--random_seed', type=int, default=2019, help='size of common item be counted')
args = parser.parse_args(args={})

In [ ]:
dataset = GMCFDataset('data/silver/', args.dataset, args.rating_file)

Processing...
(Only run at the first time training the dataset)



error number of data: 0


In [ ]:
data_num = dataset.data_N()
feature_num = dataset.feature_N()
train_index, val_index = dataset.stat_info['train_test_split_index']
print(np.concatenate((dataset[0][0:5], dataset[0][6:10])))

In [ ]:
# split inner graphs
train_dataset = dataset[:train_index]
val_dataset = dataset[train_index:val_index]
test_dataset = dataset[val_index:]

In [ ]:
n_workers = 4
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=n_workers)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=n_workers)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, num_workers=n_workers)

In [ ]:
show_loss = True
print(f"""
datast: {args.dataset}
vector dim: {args.dim}
batch_size: {args.batch_size}
lr: {args.lr}
""")


In [ ]:
datainfo = {}
datainfo['train'] = train_loader
datainfo['val'] = val_loader 
datainfo['test'] = test_loader
datainfo['feature_num'] = feature_num 
datainfo['data_num'] = [len(train_dataset), len(val_dataset), len(test_dataset)]

In [ ]:
train(args, datainfo, show_loss)